In [9]:
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

In [10]:
features = ['Age', 'SibSp', 'Parch', 'Pclass', 'Fare', 'Embarked', 'Sex']

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [11]:
# Data cleaning
train_data['Age'].fillna(train_data.Age.mean(), inplace=True)
train_data = train_data[train_data['Embarked'].notna()]

test_data['Age'].fillna(test_data.Age.mean(), inplace=True)
test_data['Embarked'].fillna('No port', inplace=True)
test_data['Fare'].fillna(test_data.Fare.mean(), inplace=True)
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          418 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         418 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


In [12]:
# Model training
y = train_data['Survived']
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

In [13]:
#Random forrest model
parameters = {
    'n_estimators': [200 * i for i in range(1, 5)],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [len(features)]
}
rfc = RandomForestClassifier()
grid = GridSearchCV(estimator=rfc, param_grid=parameters, scoring='accuracy', cv=RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1))
grid.fit(X, y)


GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=10, random_state=1),
             estimator=RandomForestClassifier(),
             param_grid={'max_depth': [7], 'max_features': ['sqrt', 'log2'],
                         'n_estimators': [200, 400, 600, 800]},
             scoring='accuracy')

In [14]:
y_pred = grid.predict(X_test)
gender = pd.read_csv('gender_submission.csv')
grid.score(X_test, gender['Survived'])

0.8899521531100478

In [15]:

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': y_pred})
output.to_csv('submission.csv', index=False)